In [12]:
import random
from IPython.display import clear_output

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

playing = True

In [13]:
class Card:
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
    
    def __str__(self):
        return self.rank + " of " + self.suit

In [14]:
class Deck:
    
    def __init__(self):
        self.deck = []  # start with an empty list
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))
    
    def __str__(self):
        deck_string = ''
        for card in self.deck:
            deck_string += "\n" + card.__str__()
        return "The deck has: "+deck_comp

    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        return self.deck.pop()

In [15]:
class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        # We are using self.aces to in a way be a boolean
        self.aces = 0    # add an attribute to keep track of aces
    
    def add_card(self,card):
        self.cards.append(card)
        self.value += values[card.rank]
        # Tracks our ace count
        if card.rank == 'Ace':
            self.aces += 1
    
    def adjust_for_ace(self):
        
        #If our total value is > 21 and I have an ace
        # Then change my ace to be a 1 instead of an 11
        while self.value > 21 and self.aces > 0:
            self.value -= 10
            #once the ace has been conveted to 1 we no longer need to keep track of it
            self.aces -= 1

In [16]:
class Chips:
    
    def __init__(self):
        self.total = 100  # This can be set to a default value or supplied by a user input
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet

In [17]:
def take_bet(chips):
    
    while True:
        
        try:
            chips.bet = int(input("How many chips would you like to bet? "))
        except:
            print("Sorry please provide a integer")
        else:
            if chips.bet > chips.total:
                print(f"Sorry you dont have enough chips! You have: {chips.total}")
            else:
                break

In [18]:
def hit(deck,hand):
    # Function takes a single card from the deck
    single_card = deck.deal()
    # then adds the card to the hand
    hand.add_card(single_card)
    # then adjusts for aces
    hand.adjust_for_ace()

In [19]:
def hit_or_stick(deck,hand):
    global playing  # to control an upcoming while loop
    
    while True:
        x = input("Hit or Stick, H or S? ")
        if x[0].lower == "h":
            hit(deck,hand)
        elif x[0].lower == "s":
            print("Dealers turn")
            playing = False
        else:
            print("Sorry I didnt understand that")
            continue
            
        break

In [20]:
#This function is used to show only some of the dealers cards

def show_some(player,dealer):
    clear_output()
    #Show only ONE of the dealers cards
    print("\nDEALERS HAND: ")
    print("First card hidden!")
    # We can use slicing as .cards is a list
    print(dealer.cards[1])
    # A easy was to print our entire hand is using a for loop
    print("\nPLAYERS HAND: ")
    for card in player.cards:
        print(card)
    print(f"Value of players hand is: {player.value}")
    
# This functiuon is to show all the cards

def show_all(player,dealer):
    clear_output()
    #Show and calculate all the dealers cards
    print("\nDEALERS HAND: ")
    for card in dealer.cards:
        print(card)
    print(f"Value of dealers hand is: {dealer.value}")
    #Show and calculate all the players cards
    print("\nPLAYERS HAND: ")
    for card in player.cards:
        print(card)
    print(f"Value of players hand is: {player.value}")

In [21]:
#Our WIN/LOSS conditions
def player_busts(player,dealer,chips):
    print("\nPLAYER BUST DEALER WINS!")
    chips.lose_bet()

def player_wins(player,dealer,chips):
    print("\nPLAYER WINS!")
    chips.win_bet()

def dealer_busts(player,dealer,chips):
    print("\nPLAYER WINS! DEALER BUST!")
    chips.win_bet()
    
def dealer_wins(player,dealer,chips):
    print("\nDEALER WINS!")
    chips.lose_bet()
    
def push(player,dealer):
    print("\nDealer and player tie!")

### And now on to the game!!

In [ ]:
while True:
    
    print("Hello and welcome to my BlackJack game! \nYou will be playing against a computer dealer and be able to place bets!")
    
 # Create & shuffle the deck, deal two cards to each player
    new_deck = Deck()
    new_deck.shuffle()
    
    player_hand = Hand()
    dealer_hand = Hand()
    
    player_hand.add_card(new_deck.deal())
    dealer_hand.add_card(new_deck.deal())
    player_hand.add_card(new_deck.deal())
    dealer_hand.add_card(new_deck.deal())
    
# Set up the Player's chips
    players_chips = Chips()
    print(f"Player has {players_chips.total} chips avalible")
# Prompt the Player for their bet
    take_bet(players_chips)

# Show cards (but keep one dealer card hidden)
    show_some(player_hand,dealer_hand)
    
    while playing:  # recall this variable from our hit_or_stand function
        
# Prompt for Player to Hit or Stand
        hit_or_stick(new_deck,player_hand)    
        
# Show cards (but keep one dealer card hidden)
        show_some(player_hand,dealer_hand)
        
# If player's hand exceeds 21, run player_busts() and break out of loop
        if player_hand.value > 21:
            player_busts(player_hand,dealer_hand,players_chips)
            break

    # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
    
    if player_hand.value <= 21:
    
        while dealer_hand.value < player_hand.value:
            hit(new_deck,dealer_hand)
        
        show_all(player_hand,dealer_hand)    
        
    # Run different winning scenarios
        if dealer_hand.value > 21:
            dealer_busts(player_hand,dealer_hand,players_chips)
        elif dealer_hand.value > player_hand.value:
            dealer_wins(player_hand,dealer_hand,players_chips)
        elif player_hand.value > dealer_hand.value:
            player_wins(player_hand,dealer_hand,players_chips)
        else:
            push(player_hand,dealer_hand)
    
    # Inform Player of their chips total 
    print(f"\nPlayer has {players_chips.total} chips avalible")   
    # Ask to play again
       
    replay = input("Would you like to play again: Y or N?")
    
    if replay == "Y":
        playing = True
        continue
    elif replay == "N":
        print("Thanks for playig!")
        break
    else:
        print("Sorry, please try again")


DEALERS HAND: 
First card hidden!
Three of Diamonds

PLAYERS HAND: 
Nine of Diamonds
Eight of Spades
Value of players hand is: 17
Hit or Stick, H or S? h
Sorry I didnt understand that
Hit or Stick, H or S? H
Sorry I didnt understand that


#### And that's it! Remember, these steps may differ significantly from your own solution. That's OK! Keep working on different sections of your program until you get the desired results. It takes a lot of time and patience! As always, feel free to post questions and comments to the QA Forums.
# Good job!